# Script-like notebook for demo
To be used to generate notes for myself and to provide a guide for the live demo. **Be sure you have extracted output files**

# Jupyter notebook
Very powerful programming environment, with support for python and other languages. Cells for this will be `python` or `markdown`, like this one. Tab completion for files, functions, and other objects.

In [ ]:
# python cell
print("Hello world")

Set up the environment. Matplotlib w/ visible figures, serpentTools

In [ ]:
import numpy
from matplotlib import pyplot, rcParams
import serpentTools

In [ ]:
rcParams['figure.figsize'] = 10, 6
rcParams['font.size'] = 16

In [ ]:
serpentTools.__version__

# Basic of `serpentTools`
Read with `serpentTools.read`. Attempts (pretty accurately) to infer file type and create the correct reader.

# ResultsFile
ResultsReader has three main locations for storing data
1. `metadata` - quantities that are invariant over transport calculations, e.g. `VERSION`
2. `resdata` - main dictionary for quanties that vary over burnup, e.g. `ABS_KEFF`
3. `universes` - dictionary that stores homogenized data, e.g. `INF_TOT`, in custom `HomogUniv` objects over burnup and across universes

In [ ]:
res = serpentTools.read('simple_res.m')

In [ ]:
res

In [ ]:
res.metadata

Most quantities are left as strings, some converted to `numpy` arrays. Names converted from `SERPENT_STYLE` to `mixedCase` to be more pythonic. 

Data accessed with brackets.

In [ ]:
res.metadata['version']

`resdata` contains a lot, so only show the keys

In [ ]:
len(res.resdata)

In [ ]:
res.resdata.keys()

In [ ]:
res.resdata['absKeff']

As a Monte Carlo code, all quantities computed by SERPENT have an associated uncertainty. These are represented as the even columns in these data, and are relative to the quantity calculated. $k_{\infty} = 1.3324 \pm 7.1\times 10^{-5}$

In [ ]:
res.resdata['totNuclides']

In [ ]:
res.resdata['anaKeff']  # total, prompt, delayed neutrons

## Homogenized Universes

In [ ]:
res.universes

In [ ]:
u0 = res.universes['0', 0, 0, 0]

In [ ]:
u0.infExp.keys()

Most group constants, except `infMicroFlx`, are stored fast groups to slow groups. `get` method retrieves data easily, with or without uncertainties

In [ ]:
u0.get('infTot')

In [ ]:
u0.get('infTot', uncertainty=True)

In [ ]:
u0.plot('infTot')

Plot labeling is passed off to `matplotlib`, which supports $\LaTeX$ labelling. Many of the plot methods allow control over log scaling of x and y axis, and confidence interval of uncertainties

In [ ]:
u0.plot(['infAbs', 'infFiss'], labels=[r'$\Sigma_a$', r'$\Sigma_f$'], logy=False)

In [ ]:
u0.plot('infMicroFlx', sigma=100)

# History Reader
Stores arrays in `arrays` dictionary. Also stores number of inactive cycles used

In [ ]:
his = serpentTools.read('hist_his0.m')

In [ ]:
his

In [ ]:
his.arrays.keys()

In [ ]:
his.numInactive

Access directly into `arrays`. Rows -> number of cycles. Columns depend on value given. Typically in groups of 3: cycle-wise mean, cumulative mean, relative uncertainty

In [ ]:
hKeff = his['anaKeff']

In [ ]:
hKeff.shape

In [ ]:
pyplot.plot(hKeff[:, 1])

Plot w/ uncertainty bands. Need x position for errorbar plot.

In [ ]:
hKu = hKeff[:, 2] * 3 * hKeff[:, 1]

In [ ]:
cyc = numpy.arange(hKu.shape[0])

In [ ]:
pyplot.plot(cyc, hKeff[:, 0], label="Cycle")
pyplot.errorbar(cyc, hKeff[:, 1], yerr=hKu, label="Cumulative")
pyplot.legend()

k is really well converged, but what about fission source?

In [ ]:
hEntr = his['entrSpt']
hEunc = hEntr[:, 2] * 3 * hEntr[:, 1]

In [ ]:
pyplot.plot(cyc, hEntr[:, 0], label="Cycle")
pyplot.errorbar(cyc, hEntr[:, 1], yerr=hEunc, label="Cumulative")
pyplot.legend()

# Detector Reader
Custom detector objects for different types of detector outputs. Stored in `detectors` dictionary, with easy accessor

In [ ]:
det = serpentTools.read('det_det0.m')

In [ ]:
det.detectors

In [ ]:
flux = det['flux']

Each detector has three attributes:
1. `bins` - 2D matrix of bin index and tally data identical to the output file
2. `tallies` - Reshaped matrix of detector tallies corresponding to bin information
3. `errors` - Reshaped matrix of detector errors corresponding to bin information

In [ ]:
flux.bins

In [ ]:
flux.bins[0, 10:]  # value, relative uncertainty

For detectors with a single bin, tallies and errors are floats

In [ ]:
flux.tallies

In [ ]:
mgf = det['mgflux']

In [ ]:
mgf.bins.shape

For detectors with "grid" information, e.g. energy, spatial meshing, etc., look at `grids` dictionary

In [ ]:
mgf.grids

In [ ]:
mgf.tallies

In [ ]:
mgf.tallies.shape, mgf.errors.shape

Plot methods with increasing flexibility and usefulness

In [ ]:
mgf.plot()

In [ ]:
mgf.plot(loglog=True)

In [ ]:
mgf.plot('energy', loglog=True, ylabel="Multi-group flux")

In [ ]:
mgf.spectrumPlot()

In [ ]:
rxrates = det['rates']

In [ ]:
rxrates.tallies.shape

In [ ]:
rxrates.indexes

Reactions are ordered as they are presented in the input file, e.g. U5 fission, U5 capture

In [ ]:
rxrates.tallies[:, 0]

`slice` method allows indexing into tally data without knowing structure *a priori*

In [ ]:
rxrates.slice(fixed={'reaction': 0})

In [ ]:
rxrates.plot(loglog=True)

In [ ]:
rxrates.plot('energy', loglog=True, labels=['Fission', r'$\gamma$'], ylabel=r'Reaction rate $\phi\Sigma$')

In [ ]:
rxrates.plot('energy', fixed={"reaction": 0}, loglog=True)

In [ ]:
xy = det['xymesh']

In [ ]:
xy.grids

In [ ]:
xy.tallies.shape

In [ ]:
xy.plot()

In [ ]:
xy.plot('xmesh')

In [ ]:
xy.meshPlot('xmesh', 'ymesh')

In [ ]:
xym = det['xymatmesh']

More complicated detector

In [ ]:
xym.tallies.shape

In [ ]:
xym.indexes

- Energy: thermal, fast
- Material: fuel, clad

In [ ]:
xym.meshPlot("xmesh", "ymesh", fixed={"energy": 0, "material": 0})

In [ ]:
xym.meshPlot("xmesh", "ymesh", fixed={"energy": 0, "material": 0}, logColor=True, cbarLabel="Fast flux")

In [ ]:
xym.meshPlot("xmesh", "ymesh", fixed={"energy": 0, "material": 1}, cbarLabel="Fast flux")

Also support reading hexagonal, cylindrical, and spherical detectors. Hex plot available for hexagonal detectors.

# Depletion Reader
Store data on `DepletedMaterial` objects in `materials` dictionary. Isotope information, `ZAI`, and burnup in `metadata`

In [ ]:
dep = serpentTools.read("dep_dep.m")

In [ ]:
dep.metadata.keys()

In [ ]:
dep.metadata['days']

In [ ]:
", ".join(dep.metadata['names'])

In [ ]:
dep.materials

In [ ]:
fuel = dep['fuel']

In [ ]:
fuel

In [ ]:
fuel.days

All arrays stored in `data` dictionary, stripped down to just the end, e.g. `MAT_FUEL_ING_TOX` -> `ingTox` stored on material `fuel`. Rows -> isotope index, columns -> time index

In [ ]:
fuel.data.keys()

In [ ]:
fuel['a'].shape

In [ ]:
u5I = fuel.names.index("U235")

In [ ]:
fuel['adens'][u5I]

In [ ]:
fuel.plot('adens', names='U235')

Plot some poison quantities

In [ ]:
fuel.plot('adens', names=['Xe135', 'Sm147', 'Sm149'])

In [ ]:
fuel.plot('burnup', 'mdens', names=['Pu239', 'U236', 'Pu240'])

Can also plot from the reader across all materials. Less exciting for this case, since there is only fuel. Read from our data files stored to make testing easier

In [ ]:
sdep = serpentTools.readDataFile('bwr_0_dep.m')

In [ ]:
sdep.materials

In [ ]:
sdep.plot('adens', names='Sm149')

# Results w/ Depletion
New data over burnup is appended to data in resdata dictionary

In [ ]:
resD = serpentTools.read('dep_res.m')

In [ ]:
resD.resdata['absKeff']

Can now showoff the plot method

In [ ]:
resD.plot({'absKeff': r'$k_{\infty}$'})

In [ ]:
resD.plot('burnStep', {'runningTime': 'Run time'}, sigma=0, right={'ompParallelFrac': "OMP Parallel Frac"})

In [ ]:
resD.universes.keys()

`getUniv` method may be more helpful for obtaining universes

In [ ]:
univD0 = resD.getUniv('0', timeDays=0)

In [ ]:
univD1 = resD.getUniv('0', index=4)

In [ ]:
univD2 = resD.getUniv('0', burnup=3.6)

In [ ]:
for univ in [univD0, univD1, univD2]:
    univ.plot("infAbs", logy=False, labels="BU: {}".format(univ.bu))
pyplot.legend()

# Depletion Matrix
Contains information that can duplicate the depletion routine from SERPENT

In [ ]:
depm = serpentTools.read('depmtx_fuelpfpr10.m')

In [ ]:
depm

In [ ]:
depm.deltaT

In [ ]:
depm.n0.shape

In [ ]:
depm.zai

Simple plot method for BOS/EOS concentrations

In [ ]:
depm.plotDensity()

In [ ]:
depm.depmtx

In [ ]:
depm.depmtx.max(), depm.depmtx.min()

Plot the structure by altering the contents. 1 if the value is positive, -1 if the value is negative

In [ ]:
depm.depmtx.data[depm.depmtx.data > 0] = 1
depm.depmtx.data[depm.depmtx.data < 0] = -1

Need to reverse the order of the rows to get the classic matrix look

In [ ]:
pyplot.pcolormesh(depm.depmtx.toarray()[::-1], cmap='PuOr_r')

# Sensitivity Reader

In [ ]:
sens = serpentTools.read('sens_sens0.m')

Maintain the ordering information with `OrderedDictionaries`

In [ ]:
sens.zais

In [ ]:
sens.materials

In [ ]:
sens.perts

In [ ]:
sens.nEne, sens.nMat, sens.nZai, sens.nPert

`sensitivities` dictionary stores the calculated sensitivities, which can be very large

In [ ]:
sens.sensitivities.keys()

In [ ]:
sens.sensitivities['keff'].shape

Material, zai, perturbation, energy, value/uncertainty.

In [ ]:
sens.energyIntegratedSens['keff'].shape

Energy index is removed as the quantity is integrated over all energy

In [ ]:
uKeffSens = sens.sensitivities['keff'][
    sens.materials['fuel'],
    (sens.zais[922350], sens.zais[922380]),
    sens.perts['total xs'],
    :,
    0,
]

In [ ]:
uKeffSens.shape

In [ ]:
pyplot.semilogx(sens.energies[1:], uKeffSens[0], label='U5')
pyplot.semilogx(sens.energies[1:], uKeffSens[1], label='U8')

Plot method plots all permutations of material, zai, and perturbation. **AVOID**. Automatically normalizes quanties per unit lethargy of the corresponding energy bin.

Plotting $\frac{\Delta k}{k}$

In [ ]:
sens.plot('keff', mat='fuel', zai=[922380, 922350], pert='total xs', labelFmt="{z} {p} in {m}")

In [ ]:
sens.plot('keff', mat='fuel', zai=[922380, 922350], pert='total xs', labelFmt="{z} {p} in {m}", normalize=False)

# Branching (coefficient) Reader
Tree-like structure storing group constants on universes across branch points

In [ ]:
coe = serpentTools.read('coe.coe')

In [ ]:
coe

In [ ]:
coe.branches.keys()

In [ ]:
bnom = coe.branches['nominal', 'nominal', 'nominal']

In [ ]:
bnom.universes

In [ ]:
buniv = bnom.universes[0, 0, 0]

In [ ]:
buniv.infExp

In [ ]:
pyplot.figure()
for [cd, ct, ft], branch in coe.branches.items():
    if cd != 'nominal':
        continue
    univ = branch.universes[0, 0, 0]
    univ.plot('infAbs', labels="{} | {}".format(ct, ft), loglog=False)
pyplot.legend(title="CT | FT")

# Branch Collector
More compact storage of group constants. Multidimensional tables over trees. Not a "standard" reader at this moment

In [ ]:
col = serpentTools.BranchCollector.fromFile('coe.coe')

In [ ]:
col.xsTables['infAbs'].shape

In [ ]:
col.axis

Without knowledge of the perturbations, difficult to determine at first how the data is structured.
Ordering is identical to input file

In [ ]:
col.burnups

In [ ]:
col.univIndex

In [ ]:
col.perturbations

In [ ]:
col.states

p0 -> coolant density, p1 -> coolant temperature, p2 -> fuel temperature. Now we see that the dimensions of the xstables are:
1. Universes : 0, 
1. Perturbation in coolant density : 0.3 g/cc, nominal 0.75 g/cc
1. Perturbation in coolant temperature : 300 K, nominal 600 K
1. Perturbation in fuel temperature: 1200 K, 300 K, nominal 900 K
1. Burnup: 0, 1 MWd/kgU
1. Energy group: 1, 2

Can change some of the attributes to be more descriptive, numeric

In [ ]:
col.perturbations = "CD", "CT", "FT"

In [ ]:
col.states = (
    numpy.array([0.3, 0.75]),
    numpy.array([300., 600, ]),
    numpy.array([1200., 300., 900.]),
)

In [ ]:
col.perturbations

In [ ]:
col.states

With this information, one can write cross section files for nodal diffusion codes with less traversal of branch trees.

# Filtering of data
By default, all data is stored during read. Can use a variety of [settings](https://serpent-tools.readthedocs.io/en/master/settingsTop.html) to control what is processed

In [ ]:
from serpentTools.settings import rc

`rc` object acts largely like a dictionary

In [ ]:
rc

In [ ]:
rc['xs.variableGroups'] = ['eig', ]
rc['xs.variableExtras'] = ['VERSION', 'INF_TOT', 'INF_S0']

In [ ]:
res2 = serpentTools.read('simple_res.m')

In [ ]:
res2.metadata

In [ ]:
res2.resdata

In [ ]:
res2.universes['0', 0, 0, 0].infExp

In [ ]:
rc['depletion.materialVariables'] = ['ADENS', ]
rc['depletion.processTotal'] = False

In [ ]:
dep2 = serpentTools.read('dep_dep.m')

In [ ]:
dep2.materials

In [ ]:
dep2['fuel'].data.keys()

# Command line interface
Rudimentary command line interface for
1. Inspecting settings
1. Creating similar input files with unique random seeds
1. Converting output files to binary `.mat` files

https://serpent-tools.readthedocs.io/en/master/command-line.html

# Python ecosystem
Many python modules that can be helpful, depending on your skillset and specific problem. Here are some that I enjoy/think will convince you to use serpentTools and python
## subprocess
Part of standard library. Can be used to execute commands inside python, like running SERPENT. Can coupled w/ serpentTools to scrape data from outputs

In [ ]:
from runner import Runner

In [ ]:
serpR = Runner('template', './sss2', sss2Opts='-omp 4')

In [ ]:
serpR.run(0.2, 0.7)

In [ ]:
rads = numpy.linspace(0.1, 0.4, 5)

In [ ]:
keffV, uncV = serpR.getRangeKeff(rads)

In [ ]:
keffV

In [ ]:
uncV *= 3 * keffV

In [ ]:
pyplot.errorbar(rads, keffV, yerr=uncV)

## scipy
Invaluable asset for science and engineering development. While numpy *largely* is used for storing data (there is vast and useful linalg support), scipy takes that further. Sparse matrices and linalg, numeric integration and differentiation, optimization, etc. With numpy, matplotlib, and scipy, ~ 80-90 % of MATLAB's core functionality and more.

Find the fuel radius that maximizes k

In [ ]:
def scalarOpt(x):
    return -serpR.run(x)[0]

In [ ]:
from scipy.optimize import minimize_scalar

In [ ]:
res = minimize_scalar(scalarOpt, method='bounded', bounds=(0.25, 0.35))

In [ ]:
pyplot.plot(rads, keffV)
pyplot.plot(res.x, -res.fun, 'x')

## pandas
Arguably the best python package (imho) for data analysis. Excellent tabulated data support, with powerful inspection routines. Can import/export between many data types (csv, excel, hdf5, etc). Includes $\LaTeX$ exporting for gorgeous tables

## seaborn
Very powerful and flexible plotting library that builds off of pandas. Can plot a large dataset against many independent parameters with a variety of options. 